# Interfacing coffea background plotting with fileGrabber json file

In [1]:
%matplotlib inline
import os
import re
import time
import json
import uproot
import awkward
import graphviz
import xgboost as xgb
import pandas as pd
import numpy as np
import utils.histoHelpers as uhh
import utils.uprootHelpers as uuh
import mvatrain.preprocessors as mpp
import matplotlib.pyplot as plt
import coffea.processor as processor
from mvatrain.metfilter import *
from awkward import JaggedArray
from os.path import join
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
from coffea.processor import defaultdict_accumulator
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from mvatrain.ROCPlot import ROCPlot
from mvatrain.hist_errorbars import hist_errorbars
from FireHydrant.Tools.uproothelpers import NestNestObjArrayToJagged
from FireHydrant.Tools.trigger import Triggers
from FireHydrant.Tools.metfilter import MetFilters
from FireHydrant.Tools.correction import get_pu_weights_function, get_ttbar_weight, get_nlo_weight_function

In [2]:
np.seterr(divide='ignore', invalid='ignore', over='ignore')
np.set_printoptions(threshold=np.inf)
TIME_STR = "190812"
TIME_STR2 = "190815"
TIME_STR_CURRENT = "190827"
DATA_DIR = join(os.environ["FFANA_BASE"], f"mvatrain/data") #base data directory
OUTPUT_DIR = join(os.environ["FFANA_BASE"], f"mvatrain/outputs/earl_grey_strong_redux")  #model
REPORT_DIR = join(os.environ["FFANA_BASE"], f"mvatrain/reports/{TIME_STR_CURRENT}") #reports and plots

In [3]:
OUTPUT_DIR_CURRENT = join(os.environ["FFANA_BASE"], f"mvatrain/outputs/earl_grey_strong_redux")
OUTPUT_DIR_MU = join(os.environ["FFANA_BASE"], f"mvatrain/outputs/english_breakfast_redux")
OUTPUT_DIR_ELEC = join(os.environ["FFANA_BASE"], f"mvatrain/outputs/irish_breakfast_redux")

print("loading model...")
xgbm_current = xgb.Booster({"nthread": 16})
xgbm_current.load_model(join(OUTPUT_DIR_CURRENT, "model_optimized/model.bin"))
if xgbm_current.attributes().get('SAVED_PARAM_predictor', None)=='gpu_predictor':
    xgbm_current.set_attr(SAVED_PARAM_predictor=None)
print("model loaded.")

print("loading model...")
xgbm_mu = xgb.Booster({"nthread": 16})
xgbm_mu.load_model(join(OUTPUT_DIR_MU, "model_optimized/model.bin"))
if xgbm_mu.attributes().get('SAVED_PARAM_predictor', None)=='gpu_predictor':
    xgbm_mu.set_attr(SAVED_PARAM_predictor=None)
print("model loaded.")

print("loading model...")
xgbm_elec = xgb.Booster({"nthread": 16})
xgbm_elec.load_model(join(OUTPUT_DIR_ELEC, "model_optimized/model.bin"))
if xgbm_elec.attributes().get('SAVED_PARAM_predictor', None)=='gpu_predictor':
    xgbm_elec.set_attr(SAVED_PARAM_predictor=None)
print("model loaded.")

loading model...
model loaded.
loading model...
model loaded.
loading model...
model loaded.


In [4]:
fill_opts = {
    'edgecolor': (0,0,0,0.3),
    'alpha': 0.8
}
error_opts = {
    'label':'Stat. Unc.',
    'hatch':'xxx',
    'facecolor':'none',
    'edgecolor':(0,0,0,.5),
    'linewidth': 0
}
data_err_opts = {
    'linestyle':'none',
    'marker': '.',
    'markersize': 10.,
    'color':'k',
    'elinewidth': 1,
    'emarker': '_'
}

In [5]:
datasets_ = json.load(open("2018.json"))

bkgdatasets = {}

for group in datasets_.keys():
    if "DoubleMuon" in group: continue
    elif "CRAB_PrivateMC" in group: continue
    else:
        cutgroup = group[:-5]
        if cutgroup == 'DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_':
            cutgroup = cutgroup[:-1]
        if cutgroup not in bkgdatasets.keys(): bkgdatasets[cutgroup] = {'files': [], 'treename': ""}
        bkgdatasets[cutgroup]['files'].extend(datasets_[group]['files'])
        bkgdatasets[cutgroup]['treename'] = 'ffNtuplizer/ffNtuple'
        
counter = 0
        
bkgscales = {}
for group in datasets_.keys():
    if "DoubleMuon" in group: continue
    elif "CRAB_PrivateMC" in group: continue
    else:
        if counter == 0:
            cutgroup = group[:-5]
            if cutgroup == 'DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_':
                cutgroup = cutgroup[:-1]
            bkgscales[cutgroup] = datasets_[group]['weight']
            previous = cutgroup
        else:
            if datasets_[group]['weight'] != bkgscales[previous]:
                cutgroup = group[:-5]
                if cutgroup == 'DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8_':
                    cutgroup = cutgroup[:-1]
                bkgscales[cutgroup] = datasets_[group]['weight']
                previous = cutgroup
                
bkgmapping = {}
for k in bkgdatasets: bkgmapping[k] = list(bkgdatasets[k])
    
datadatasets = {}
for group in datasets_.keys():
    if "DoubleMuon" not in group: continue
    cutgroup = group[:-5]
    if cutgroup == "DoubleMuon_":
        cutgroup = cutgroup[:-1]
    if "Run2018A" in cutgroup:
        datadatasets['A']['files'] = []
        datadatasets['A']['files'].extend(datasets_[group]['files'])
        datadatasets['A']['treename'] = 'ffNtuplizer/ffNtuple'
    elif "Run2018B" in cutgroup:
        datadatasets['B']['files'] = []
        datadatasets['B']['files'].extend(datasets_[group]['files'])
        datadatasets['B']['treename'] = 'ffNtuples/ffNtuple'
    elif "Run2018C" in cutgroup:
        datadatasets['C']['files'] = []
        datadatasets['C']['files'].extend(datasets_[group]['files'])
        datadatasets['C']['treename'] = 'ffNtuples/ffNtuple'
    elif "Run2018D" in cutgroup:
        datadatasets['D']['files'] = []
        datadatasets['D']['files'].extend(datasets_[group]['files'])
        datadatasets['D']['treename'] = 'ffNtuples/ffNtuple'
        
datamapping = {'data': list('ABCD')}

datasets = {}
datasets.update(bkgdatasets)
datasets.update(datadatasets)

mapping = {}
mapping.update(bkgmapping)
mapping.update(datamapping)

notdata = re.compile('(?!data)')

In [6]:
class LeptonJetsProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat('dataset', 'Control region')
        pt_axis       = hist.Bin("pt", "pT [GeV]", 50, 0, 800)
        eta_axis      = hist.Bin("eta", 'eta', 50, -2.4, 2.4)
        neufrac_axis      = hist.Bin("neufrac", "neutral energy fraction", 50, 0, 1.001)
        maxd0_axis    = hist.Bin("maxd0", 'track max |d0|', 50, 0, 0.5)
        mind0_axis    = hist.Bin("mind0", 'track min |d0|', 50, 0, 0.5)
        tkiso_axis    = hist.Bin('tkiso', 'track isolation', 50, 0, 1)
        pfiso_axis    = hist.Bin("pfiso", "PFCands isolation", 50, 0, 1)
        spreadpt_axis = hist.Bin("spreadpt", "spreadpt", 50, 0, 1)
        spreaddr_axis = hist.Bin("spreaddr", "spreaddr", 50, 0, 0.1)
        lambda_axis   = hist.Bin('lambda', 'jet sub - lambda', 50, -8, 0)
        epsilon_axis  = hist.Bin('epsilon', 'jet sub - epsilon', 50, 0, 0.25)
        ecf1_axis    = hist.Bin('ecf1', 'energy correlation function - e1', 50, 0, 200)
        ecf2_axis    = hist.Bin('ecf2', 'energy correlation function - e2', 50, 0, 500)
        ecf3_axis    = hist.Bin('ecf3', 'energy correlation function - e3', 50, 0, 300)
        mva_axis      = hist.Bin('mva', 'BDT value', 50, -10, 10)
        
        self._accumulator = processor.dict_accumulator({
            'pt': hist.Hist("#counts/16GeV", dataset_axis, pt_axis),
            "eta": hist.Hist("#counts/0.096", dataset_axis, eta_axis),
            "neufrac": hist.Hist("#counts/0.02", dataset_axis, neufrac_axis),
            "maxd0": hist.Hist("#counts/0.01cm", dataset_axis, maxd0_axis),
            "mind0": hist.Hist("#counts/0.01cm", dataset_axis, mind0_axis),
            "tkiso": hist.Hist("#counts/0.02", dataset_axis, tkiso_axis),
            "pfiso": hist.Hist("#counts/0.02", dataset_axis, pfiso_axis),
            "spreadpt": hist.Hist("#counts/0.02", dataset_axis, spreadpt_axis),
            "spreaddr": hist.Hist("#counts/0.002", dataset_axis, spreaddr_axis),
            "lambda": hist.Hist("#counts/0.16", dataset_axis, lambda_axis),
            "epsilon": hist.Hist("#counts/0.005", dataset_axis, epsilon_axis),
            "ecf1": hist.Hist("#counts/4", dataset_axis, ecf1_axis),
            "ecf2": hist.Hist("#counts/10", dataset_axis, ecf2_axis),
            "ecf3": hist.Hist("#counts/6", dataset_axis, ecf3_axis),
            "mva": hist.Hist("#counts/0.4", dataset_axis, mva_axis),
        })
        self.pucorrs = get_pu_weights_function()
        self.nlo_w = get_nlo_weight_function('w')
        self.nlo_z = get_nlo_weight_function('z')

    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        
        dataset = df['dataset']
        if dataset.startswith('TTJets_'): return output # skip TTJets_
        
        ## construct weights ##
        wgts = processor.Weights(df.size)
        
        metfiltermask = np.logical_and.reduce([df[mf] for mf in MetFilters])
        wgts.add('metfilters', metfiltermask)
        
        if len(dataset)!=1:
            wgts.add('genw', df['weight'])

            nvtx = df['trueInteractionNum']
            pu, puUp, puDown = (f(nvtx) for f in self.pucorrs)
            wgts.add('pileup', pu, puUp, puDown)

            wnlo = np.ones_like(df.size)
            if 'TTJets' in dataset or 'WJets' in dataset or 'DYJets' in dataset:
                genparticles = JaggedCandidateArray.candidatesfromcounts(
                    df['gen_p4'],
                    px=df['gen_p4.fCoordinates.fX'],
                    py=df['gen_p4.fCoordinates.fY'],
                    pz=df['gen_p4.fCoordinates.fZ'],
                    energy=df['gen_p4.fCoordinates.fT'],
                    pid=df['gen_pid'],
                )
                gentops = genparticles[np.abs(genparticles.pid)==6]
                genws = genparticles[np.abs(genparticles.pid)==24]
                genzs = genparticles[np.abs(genparticles.pid)==23]

                if 'TTJets' in dataset:
                    wnlo = np.sqrt(get_ttbar_weight(gentops[0].p4.pt.sum()) * get_ttbar_weight(gentops[1].p4.pt.sum()))
                elif 'WJets' in dataset:
                    wnlo = self.nlo_w(genws[0].p4.pt.sum())
                elif 'DYJets' in dataset:
                    wnlo = self.nlo_z(genzs[0].p4.pt.sum())
            wgts.add('nlo', wnlo)
            
        weight = wgts.weight()
        ########################
        
        absd0 = np.abs(NestNestObjArrayToJagged(df['pfjet_pfcand_tkD0'])).fillna(0)
        
        leptonjets = JaggedCandidateArray.candidatesfromcounts(
            df['pfjet_p4'], **{
            "px": df['pfjet_p4.fCoordinates.fX'],
            "py": df['pfjet_p4.fCoordinates.fY'],
            "pz": df['pfjet_p4.fCoordinates.fZ'],
            "energy": df['pfjet_p4.fCoordinates.fT'],
            "neufrac": ((df['pfjet_neutralEmE']+df['pfjet_neutralHadronE'])/df['pfjet_p4.fCoordinates.fT']),
            "maxd0": absd0.max().content,
            "mind0": absd0.min().content,
            "tkiso": df['pfjet_tkIsolation05'],
            "pfiso": df['pfjet_pfIsolation05'],
            "spreadpt": df['pfjet_ptDistribution'],
            "spreaddr": df['pfjet_dRSpread'],
            "lambda": df['pfjet_subjet_lambda'],
            "epsilon": df['pfjet_subjet_epsilon'],
            "ecf1": df['pfjet_subjet_ecf1'],
            "ecf2": df['pfjet_subjet_ecf2'],
            "ecf3": df['pfjet_subjet_ecf3'],
            })
        
        vals={
            'target': leptonjets.pt.zeros_like().flatten(),
            'pt': leptonjets.pt.flatten(),
            'eta': leptonjets.eta.flatten(),
        }
        
        vals.update( {k: leptonjets[k].flatten() for k in output.keys() if (k not in vals and k != 'mva')} )
        
        dfleptonjets = pd.DataFrame(vals)
        
        dfleptonjets.fillna(0)
        
        feature_cols = [n for n in dfleptonjets.keys() if n != "target"]

        dfleptonjets2 = dfleptonjets[feature_cols]
        
        xglj = xgb.DMatrix(dfleptonjets2)
        
        predictions = xgbm_current.predict(xglj)
        
        offsets = leptonjets.pt.offsets
        jaggedpredictions = JaggedArray.fromoffsets(offsets, predictions)
        leptonjets.add_attributes(mva=jaggedpredictions)
        
        twoleptonjets = leptonjets.counts>=2
        dileptonjets = leptonjets[twoleptonjets]
        wgt = weight[twoleptonjets]
        
        leptonjetpair = dileptonjets.distincts()
        sumpt = leptonjetpair.i0.pt+leptonjetpair.i1.pt
        if sumpt.size==0: return output
        
        leadingLjPair = leptonjetpair[sumpt.argmax()]
        controlregion = np.abs(leadingLjPair.i0.p4.delta_phi(leadingLjPair.i1.p4))<2.5
        leptonjets_ = dileptonjets[controlregion.flatten()]
        wgt = wgt[controlregion.flatten()]
        wgts= (leptonjets_.pt.ones_like()*wgt).flatten()

        output['pt'].fill(dataset=dataset, pt=leptonjets_.pt.flatten(), weight=wgts)
        output['eta'].fill(dataset=dataset, eta=leptonjets_.eta.flatten(), weight=wgts)
        output['neufrac'].fill(dataset=dataset, neufrac=leptonjets_.neufrac.flatten(), weight=wgts)
        output['maxd0'].fill(dataset=dataset, maxd0=leptonjets_.maxd0.flatten(), weight=wgts)
        output['mind0'].fill(dataset=dataset, mind0=leptonjets_.mind0.flatten(), weight=wgts)
        output['tkiso'].fill(dataset=dataset, tkiso=leptonjets_.tkiso.flatten(), weight=wgts)
        output['pfiso'].fill(dataset=dataset, pfiso=leptonjets_.pfiso.flatten(), weight=wgts)
        output['spreadpt'].fill(dataset=dataset, spreadpt=leptonjets_.spreadpt.flatten(), weight=wgts)
        output['spreaddr'].fill(dataset=dataset, spreaddr=leptonjets_.spreaddr.flatten(), weight=wgts)
        output['lambda']  .fill(**{"dataset": dataset, "lambda": leptonjets_['lambda'].flatten(), 'weight': wgts})
        output['epsilon'].fill(dataset=dataset, epsilon=leptonjets_.epsilon.flatten(), weight=wgts)
        output['ecf1'].fill(dataset=dataset, ecf1=leptonjets_.ecf1.flatten(), weight=wgts)
        output['ecf2'].fill(dataset=dataset, ecf2=leptonjets_.ecf2.flatten(), weight=wgts)
        output['ecf3'].fill(dataset=dataset, ecf3=leptonjets_.ecf3.flatten(), weight=wgts)
        output['mva'].fill(dataset=dataset, mva=leptonjets_.mva.flatten(), weight=wgts)
        
        return output
    
    def postprocess(self, accumulator):
        origidentity = list(accumulator)
        
        for k in origidentity:
            # scale
            accumulator[k].scale(bkgscales, axis='dataset')
            # cat grouping
            accumulator[k+'_cat'] = accumulator[k].group(hist.Cat("cat", "datasets"), "dataset", mapping)
        
        return accumulator

In [8]:
output = processor.run_uproot_job(datasets,
                                  treename=None,
                                  processor_instance=LeptonJetsProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing:   2%|▏         | 55/2306 [00:03<02:07, 17.72items/s]


PicklingError: Can't pickle <class 'KeyError'>: attribute lookup _KeyError on builtins failed